In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle

X = pickle.load(open("X_2.pickle", "rb"))
y = pickle.load(open("y_2.pickle", "rb"))

X = X/255

model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy",
              optimizer="adam", 
              metrics=['accuracy'])

model.fit(X, y, batch_size=100, epochs=20, validation_split=.1)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 900 samples, validate on 100 samples
Epoch 1/20
900/900 [==============================] - 55s 61ms/sample - loss: 2.1371 - acc: 0.5033 - val_loss: 0.6684 - val_acc: 0.4900
Epoch 2/20
900/900 [==============================] - 50s 56ms/sample - loss: 0.4993 - acc: 0.7867 - val_loss: 0.4070 - val_acc: 0.8000
Epoch 3/20
900/900 [==============================] - 50s 56ms/sample - loss: 0.2344 - acc: 0.9222 - val_loss: 0.1640 - val_acc: 0.9500
Epoch 4/20
900/900 [==============================] - 50s 56ms/sample - loss: 0.0870 - acc: 0.9711 - val_loss: 0.1048 - val_acc: 0.9600
Epoch 5/20
900/900 [==============================] - 50s 56ms/sample - loss: 0.0403 - acc: 0.9911 - val_loss: 0.0618 - val_acc: 0.9800
Epoch 6/20
900/900 [==============================] - 53s 58ms

In [6]:
import os
os.makedirs('./model', exist_ok=True)
model.save('./model/te_cnn_model_v3.h5')

CONVERT FROM .h5 to .pb

In [7]:
from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

Using TensorFlow backend.


In [8]:
import tensorflow as tf 
model = tf.keras.models.load_model('./model/te_cnn_model_v3.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 297, 297, 64)      640       
_________________________________________________________________
activation (Activation)      (None, 297, 297, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 148, 148, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 64)      36928     
_________________________________________________________________
activation_1 (Activation)    (None, 146, 146, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 341056)            0

In [10]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

In [11]:
from keras import backend as K
import tensorflow as tf

frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 58 variables.
INFO:tensorflow:Converted 58 variables to const ops.


In [12]:
tf.train.write_graph(frozen_graph, "model", "te_cnn_model_V3.pb", as_text=False)

'model\\te_cnn_model_V3.pb'